## Segmentación de TACs cerebrales de pacientes con ictus utilizando Niftynet y la configuración y red entrenada de los TACs abdominales.

**Autor**: Bárbara Sainz Crespo, *Ingeniería Informática, Escuela Politécnica Superior, Universidad de Burgos*.

**Fecha**: 2019/06/10

**Descripción**: Prueba de segmentación de nuestras imágenes de TACs cerebrales de pacientes con ictus, para la identificación ROIs (regiones de interés), con la configuración inicial comprobada para TACs abdominales. Para ello, primero se ubican los datos de nuestras propias imágenes TAC en su path correspondiente, para interaccionar con el modelo y los pesos del entrenamiento previo. Entre estos últimos archivos se incluye el fichero de configuración inicial, para llevar a cabo el proceso. Después, se segmentan los datos introducidos, para ver el resultado de la identificación de ROIs con este modelo.

In [34]:
# Guardamos las imágenes de TACs abdominales en otra carpeta, dejando libre la utilizada para los datos de entrada del sistema.
! mv ~/niftynet/data/dense_vnet_abdominal_ct/*.nii ~/niftynet/abd_CTs/
! ls ~/niftynet/abd_CTs/
! echo "Esta carpeta ya debería estar vacía: "
! ls ~/niftynet/data/dense_vnet_abdominal_ct/

100_CT.nii  100_Label.nii
Esta carpeta ya debería estar vacía: 


In [44]:
# Ubicar la imagen del TC y su máscara (label), lo que queremos segmentar, en el directorio de entrada de datos del sistema.
! cp ~/niftynet/TCs_masked/TC001.nii ~/niftynet/data/dense_vnet_abdominal_ct/'001_CT.nii'
! cp ~/niftynet/TCs_masked/TC001_mask.nii.gz ~/niftynet/data/dense_vnet_abdominal_ct/'001_Label.nii.gz'
# Nuestras máscaras están comprimidas (.nii.gz), por ello hay que descomprimirlas, ya que el sistema recibe ficheros ".nii".
! gzip -d ~/niftynet/data/dense_vnet_abdominal_ct/'001_Label.nii.gz'
# Comprobar que el TC y su máscara están correctamente ubicados y preparados para la segmentación.
! ls ~/niftynet/data/dense_vnet_abdominal_ct/

001_CT.nii  001_Label.nii


In [46]:
# Los ficheros se modificaron para que contengan "TC" y "Label" respectivamente, como busca el archivo de configuración inicial.
! cat ~/niftynet/extensions/dense_vnet_abdominal_ct/config.ini


############################ input configuration sections
[ct]
path_to_search = ./data/dense_vnet_abdominal_ct/
filename_contains = CT
spatial_window_size = (144, 144, 144)
interp_order = 1
axcodes=(A, R, S)

[label]
path_to_search = ./data/dense_vnet_abdominal_ct/
filename_contains = Label
spatial_window_size = (144, 144, 144)
interp_order = 0
axcodes=(A, R, S)

############################## system configuration sections
[SYSTEM]
cuda_devices = ""
num_threads = 1
num_gpus = 1
model_dir = models/dense_vnet_abdominal_ct
queue_length = 36

[NETWORK]
name = dense_vnet
# batch size 1 for inference
# batch size 6 for training
batch_size = 1

# volume level preprocessing
volume_padding_size = 0
window_sampling = resize

[TRAINING]
sample_per_volume = 1
lr = 0.001
loss_type = dense_vnet_abdominal_ct.dice_hinge.dice
starting_iter = 0
save_every_n = 1000
max_iter = 3001

[INFERENCE]
border = (0, 0, 0)
inference_iter = 3000
output_interp_order = 0
spatial_window_size = (144, 144, 144)
save_seg_

In [47]:
# Segmentar utilizando nuestros datos ya preparados, desde la configuración inicial previamente mostrada.
! net_segment inference -c ~/niftynet/extensions/dense_vnet_abdominal_ct/config.ini

CRITICAL:tensorflow:Optional Python module cv2 not found, please install cv2 and retry if the application fails.
CRITICAL:tensorflow:Optional Python module skimage.io not found, please install skimage.io and retry if the application fails.
CRITICAL:tensorflow:Optional Python module SimpleITK not found, please install SimpleITK and retry if the application fails.
INFO:niftynet: Optional Python module yaml not found, please install yaml and retry if the application fails.
INFO:niftynet: Optional Python module yaml version None not found, please install yaml-None and retry if the application fails.
INFO:niftynet: Optional Python module SimpleITK not found, please install SimpleITK and retry if the application fails.
INFO:niftynet: Optional Python module SimpleITK version None not found, please install SimpleITK-None and retry if the application fails.
NiftyNet version 0.5.0
INFO:niftynet: Optional Python module yaml not found, please install yaml and retry if the application fails.
INFO:n

2019-06-10 20:23:41.390131: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1490] Adding visible gpu devices: 0, 1, 2
2019-06-10 20:23:41.390299: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] Device interconnect StreamExecutor with strength 1 edge matrix:
2019-06-10 20:23:41.390312: I tensorflow/core/common_runtime/gpu/gpu_device.cc:977]      0 1 2 
2019-06-10 20:23:41.390320: I tensorflow/core/common_runtime/gpu/gpu_device.cc:990] 0:   N Y Y 
2019-06-10 20:23:41.390326: I tensorflow/core/common_runtime/gpu/gpu_device.cc:990] 1:   Y N Y 
2019-06-10 20:23:41.390332: I tensorflow/core/common_runtime/gpu/gpu_device.cc:990] 2:   Y Y N 
2019-06-10 20:23:41.390660: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1103] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10907 MB memory) -> physical GPU (device: 0, name: TITAN Xp, pci bus id: 0000:05:00.0, compute capability: 6.1)
2019-06-10 20:23:41.390901: I tensorflow/core/common_runtime/gpu/gpu_devi

In [49]:
# Comprobar que tenemos el resultado de la segmentación en el directorio creado para el output.
! echo "La máscara creada por la segmentación debería aparecer como '001__niftynet_out.nii.gz'"
! ls ~/niftynet/models/dense_vnet_abdominal_ct/segmentation_output

La máscara creada por la segmentación debería aparecer como '001__niftynet_out.nii.gz'
001__niftynet_out.nii.gz  100__niftynet_out.nii.gz  inferred.csv
